In [2]:
from openai import OpenAI
client = OpenAI()

# assistant = client.beta.assistants.create(
#   name="Tutor",
#   instructions="You are a personal tutor. Teach the user on the given topic and if files are provided then use them and then teach or answer the question.",
#   tools=[{"type": "file_search"}],
#   model="gpt-4o",
# )
# assistant.to_dict()

In [10]:
files = client.files.list()
files.data

[FileObject(id='file-2tCLC6uS9nJVoDLPdwhv4o', bytes=18134359, created_at=1739102176, filename='The Deck.pdf', object='file', purpose='assistants', status='processed', status_details=None),
 FileObject(id='file-6GBtLz5g8gcwkVZ3gSWhWy', bytes=30377, created_at=1739102169, filename='capacity-retrieval-agent (2).html', object='file', purpose='assistants', status='processed', status_details=None)]

In [7]:
for i in files.data:
    if i.id == '':
        pass
    else:
        client.files.delete(i.id)

In [2]:
thread = client.beta.threads.create()

In [3]:
message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content="I need to solve the equation `3x + 11 = 14`. Can you help me?"
)

In [4]:
from typing_extensions import override
from openai import AssistantEventHandler
 
# First, we create a EventHandler class to define
# how we want to handle the events in the response stream.
 
class EventHandler(AssistantEventHandler):    
  @override
  def on_text_created(self, text) -> None:
    print(f"\nassistant > ", end="", flush=True)
      
  @override
  def on_text_delta(self, delta, snapshot):
    print(delta.value, end="", flush=True)
      
  def on_tool_call_created(self, tool_call):
    print(f"\nassistant > {tool_call.type}\n", flush=True)
  
  def on_tool_call_delta(self, delta, snapshot):
    if delta.type == 'code_interpreter':
      if delta.code_interpreter.input:
        print(delta.code_interpreter.input, end="", flush=True)
      if delta.code_interpreter.outputs:
        print(f"\n\noutput >", flush=True)
        for output in delta.code_interpreter.outputs:
          if output.type == "logs":
            print(f"\n{output.logs}", flush=True)
 
# Then, we use the `stream` SDK helper 
# with the `EventHandler` class to create the Run 
# and stream the response.
 
with client.beta.threads.runs.stream(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please address the user as Jane Doe. The user has a premium account.",
  event_handler=EventHandler(),
) as stream:
  stream.until_done()


assistant > Of course, Jane Doe! To solve the equation \(3x + 11 = 14\), we need to isolate \(x\). Let's do it step by step:

1. Subtract 11 from both sides of the equation:
   \[
   3x + 11 - 11 = 14 - 11
   \]
   \[
   3x = 3
   \]
   
2. Divide both sides by 3:
   \[
   x = \frac{3}{3} = 1
   \]

Thus, the solution to the equation is \(x = 1\).

# Deep Dive

In [6]:
file = client.files.create(
  file=open("/home/ub-karar/Downloads/Revenue_Forecast.csv", "rb"),
  purpose='assistants'
)

In [9]:
file.to_dict()

{'id': 'file-QW3kZpBkcUx13ChdymxrNa',
 'bytes': 2046,
 'created_at': 1738930331,
 'filename': 'Revenue_Forecast.csv',
 'object': 'file',
 'purpose': 'assistants',
 'status': 'processed',
 'status_details': None}

In [10]:
assistant = client.beta.assistants.create(
  name="Data visualizer",
  description="You are great at creating beautiful data visualizations. You analyze data present in .csv files, understand trends, and come up with data visualizations relevant to those trends. You also share a brief text summary of the trends observed.",
  model="gpt-4o",
  tools=[{"type": "code_interpreter"}],
  tool_resources={
    "code_interpreter": {
      "file_ids": [file.id]
    }
  }
)

In [12]:
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "Create 3 data visualizations based on the trends in this file.",
      "attachments": [
        {
          "file_id": file.id,
          "tools": [{"type": "code_interpreter"}]
        }
      ]
    }
  ]
)

In [13]:
with client.beta.threads.runs.stream(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please address the user as Jane Doe. The user has a premium account.",
  event_handler=EventHandler(),
) as stream:
  stream.until_done()


assistant > Sure, Jane Doe. I'll start by examining the contents of the uploaded file to understand the data. Then, I'll create three visualizations based on trends within the data. Let's begin by loading and exploring the file.
assistant > code_interpreter

import pandas as pd

# Load the uploaded file to examine its contents
file_path = '/mnt/data/file-QW3kZpBkcUx13ChdymxrNa'
data = pd.read_csv(file_path)

# Display the first few rows of the dataframe and its structure
data.head(), data.info()

output >

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Fiscal_Year       32 non-null     object 
 1   Fiscal_Year_End   32 non-null     int64  
 2   Revenue_Type      32 non-null     object 
 3   Variance_Percent  32 non-null     float64
 4   Date              32 non-null     object 
 5   ObjectId          32 non-null     int64  
dtypes: f

---

- We only need file upload - no vectorDB is required
- Add attachments id in assistent message

---

In [15]:
from openai import OpenAI
client = OpenAI()

my_assistants = client.beta.assistants.list(
    order="desc",
    limit="20",
)
print(my_assistants.data)
len(my_assistants.data)

[Assistant(id='asst_ksR01Wxi2WSpZ7GPHcefjMD7', created_at=1739102700, description=None, instructions='You are a personal tutor. Teach the user on the given topic and if files are provided then use them and then teach or answer the question.', metadata={}, model='gpt-4o', name='Tutor', object='assistant', tools=[FileSearchTool(type='file_search', file_search=FileSearch(max_num_results=None, ranking_options=FileSearchRankingOptions(score_threshold=0.0, ranker='default_2024_08_21')))], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=ToolResourcesFileSearch(vector_store_ids=[])), top_p=1.0, reasoning_effort=None)]


1

In [12]:
for i in my_assistants.data:
    print(i.id)
    client.beta.assistants.delete(i.id)


asst_hWe5V6f3MYQcsghg9xuHFqCI


In [1]:
from openai import OpenAI
client = OpenAI()

thread_message = client.beta.threads.messages.create(
  "thread_Mijyb8qS2J6J9m9UbqKeePAU",
  role="user",
  content="please agin tell the summar of the file.",
)
print(thread_message)


Message(id='msg_QNii1Bj79TD2UXVFjEAZF66i', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='please agin tell the summar of the file.'), type='text')], created_at=1739134872, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_Mijyb8qS2J6J9m9UbqKeePAU')


In [2]:
run = client.beta.threads.runs.create(
  thread_id="thread_Mijyb8qS2J6J9m9UbqKeePAU",
  assistant_id="asst_ksR01Wxi2WSpZ7GPHcefjMD7"
)

print(run)

Run(id='run_GFGnIVIf5dtPFDNoeWq6VukW', assistant_id='asst_ksR01Wxi2WSpZ7GPHcefjMD7', cancelled_at=None, completed_at=None, created_at=1739135004, expires_at=1739135604, failed_at=None, incomplete_details=None, instructions='You are a personal tutor. Teach the user on the given topic and if files are provided then use them and then teach or answer the question.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_Mijyb8qS2J6J9m9UbqKeePAU', tool_choice='auto', tools=[FileSearchTool(type='file_search', file_search=FileSearch(max_num_results=None, ranking_options=FileSearchRankingOptions(score_threshold=0.0, ranker='default_2024_08_21')))], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=None, temperature=1.0, top_p=1.0, tool_resources={}, reasoning_effort=None)


In [11]:
thread_messages = client.beta.threads.messages.list("thread_ZvewSKOMBjP8GBAMm6wu5df7")
print(len(thread_messages.data))
for i in thread_messages.data:
    print(i.role, i.content)

1
user [TextContentBlock(text=Text(annotations=[], value='summrize this file'), type='text')]
